In [ ]:
# !pip install azure-ai-ml
# !pip install azure-identity
# !pip install azureml-inference-server-http

In [ ]:
# !az login

In [ ]:
# Before creating deployment in Azure, you need to test/debug it locally.
# local deployment must have Docker and Azure CLI v2 installed on your local machine

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from datetime import datetime
import os
import tempfile
from azure.ai.ml.entities import (ManagedOnlineEndpoint,
                                  ManagedOnlineDeployment,
                                  Model,
                                  Environment,
                                  CodeConfiguration
                                  )
from training_scripts.utils.logging import Logger

subscription_id = '' # update subscription id
resource_group = '' # update ml resource group
workspace = '' # update ml workspace name

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

In [7]:
# add model.bin and conda_env.yaml to current directory in local machine
# model.bin can  be found under artifacts section of current registered model. Download it
# environment yaml can be found in latest registered env. Copy the yaml code and paste it in conda_env.yaml in local machine 
latest_model = Model(path="model.bin") 
latest_env = Environment(conda_file="conda_env.yaml",
                         image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest"
                        )

In [8]:
# endpoint name should be unique in region
endpoint_name = "endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(name = endpoint_name, description="my endpoint",auth_mode="key")

In [9]:
deployment = ManagedOnlineDeployment(name="my-deployment",
                                     endpoint_name=endpoint_name,
                                     model=latest_model,
                                     environment=latest_env,
                                     code_configuration=CodeConfiguration(code=".", scoring_script="scoring_script.py"),
                                     instance_type="Standard_DS3_v2",
                                     instance_count=1,
                                     environment_variables={'CLOUDPICKLE_MODEL_NAME_TO_SAVE': "model.bin"}
                                    )

In [ ]:
# create or update deployment with local=True for local deployment 
ml_client.online_endpoints.begin_create_or_update(endpoint=endpoint, local=True)

In [ ]:
# 'provisioning_state': 'Succeeded' means deployment is successfully created
ml_client.online_deployments.get(name="my-deployment", endpoint_name=endpoint_name, local=True)

In [118]:
# create sample.json with payload in current dir of local machine
response = ml_client.online_endpoints.invoke(endpoint_name=endpoint_name,
                                             request_file="sample.json",
                                             local=True
                                            )
print(response)
print(type(response))

{"input": ["123abc$%%$%gGG", "1QW!@#asQQ", "Adf^%$d1"], "strength": ["Very strong", "Strong", "Strong"], "confidence": [1.0, 1.0, 1.0], "status": 200, "message": ""}
<class 'str'>


In [ ]:
# print the logs of deployment
print(ml_client.online_deployments.get_logs(name="my-deployment", 
                                            endpoint_name=endpoint_name, 
                                            local=True, lines=500
                                           ))